In [1]:
class memoize(object): # pylint: disable=invalid-name
    """Decorator to memoize a function"""
    def __init__(self, func):
        self.func = func
        self.cache = {}
        
    def __get__(self, instance, owner):
        self.parent = instance
        print('GET', instance, owner)
        return self.__call__

    def __call__(self, *args):
        if args not in self.cache:
            self.cache[args] = self.func(*args)
        return self.cache[args]
    
@memoize
def foobar(ducks=10, guns=0):
    return ducks - guns


print(foobar.__dict__)

remaining = foobar(10, 3)
print(remaining)

print(foobar.__dict__)

{'cache': {}, 'func': <function foobar at 0x7f08f9486de8>}
7
{'cache': {(10, 3): 7}, 'func': <function foobar at 0x7f08f9486de8>}


In [1]:
from functools import wraps
from inspect import signature

def memoize(f):
    print('init decorator',f)
    
    def get_first_param(f):
        sig = signature(f)
        if len(sig.parameters) < 1:
            return None
        return list(sig.parameters.keys())[0]
    
    fp = get_first_param(f)
    # save cache in the parent object
    def class_memoize(f):
        print('init cls', f)
        # wrapper class
        @wraps(f)
        def wrapper(*args):
            print('args', args)
            cache = getattr(args[0], '_cache', {})
            print('F', f, f.__name__)
            key = (f.__name__, args[1:])
            if key not in cache:
                cache[key] = f(*args)
                setattr(args[0], '_cache', cache)
            return cache[key]
        return wrapper
    # save cache in the function object itself
    def func_memoize(f):
        print('init func', f)
        cache = getattr(f, 'cache', {})
        print('FM CACHE', cache, f)
        
        @wraps(f)
        def wrapper(*args):
            print('ARGS', args)
            cache = getattr(f, 'cache', {})
            print('CACHE', cache, f.__dict__)
            print('inside wrapper', f, )
            result = f(*args)
            if args not in cache:
                cache[args] = result
                setattr(f, 'cache', cache)
            return result
        return wrapper
    # decide which one to give
    if fp == 'self':
        print('CLASS MEMOIZE')
        return class_memoize(f)
    print('FUNC MEMOIZE')
    return func_memoize(f)

    
    

class DUCKS:
    
    def __init__(self, ducks):
        print('INIT DUCKS', ducks)
        self.ducks = ducks
    
    @property
    @memoize
    def count(self):
        return self.ducks

    @memoize
    def hunt(self, guns=1):
        return self.ducks - guns

@memoize    
def foobar(ducks, guns):
    return ducks - guns

@memoize 
def foobaz():
    return 42


d = DUCKS(20)

print('---COUNT---')
d = DUCKS(32)
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---COUNT---')
print('dict:', d.__dict__)
r = d.count
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(12)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---HUNT---')
print('dict:', d.__dict__)
r = d.hunt(13)
print('result:', r)
print('dict:', d.__dict__)
print()

print('---FOOBAR---')
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
r = foobar(12, 3)
print('result:', r)
print('foobar:', foobar)
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
print()

print('---FOOBAR---')
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
r = foobar(12, 3)
print('result:', r)
print('foobar:', foobar)
print('dict:', foobar.__closure__[0].cell_contents.__dict__)
print()


print('---FOOBAZ---')
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
r = foobaz()
print('result:', r)
print('foobar:', foobaz)
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
print()


print('---FOOBAZ---')
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
r = foobaz()
print('result:', r)
print('foobar:', foobaz)
print('dict:', foobaz.__closure__[0].cell_contents.__dict__)
print()

init decorator <function DUCKS.count at 0x7fd10c132f28>
CLASS MEMOIZE
init cls <function DUCKS.count at 0x7fd10c132f28>
init decorator <function DUCKS.hunt at 0x7fd10c132b70>
CLASS MEMOIZE
init cls <function DUCKS.hunt at 0x7fd10c132b70>
init decorator <function foobar at 0x7fd10e2212f0>
FUNC MEMOIZE
init func <function foobar at 0x7fd10e2212f0>
FM CACHE {} <function foobar at 0x7fd10e2212f0>
init decorator <function foobaz at 0x7fd10c132ae8>
FUNC MEMOIZE
init func <function foobaz at 0x7fd10c132ae8>
FM CACHE {} <function foobaz at 0x7fd10c132ae8>
INIT DUCKS 20
---COUNT---
INIT DUCKS 32
dict: {'ducks': 32}
args (<__main__.DUCKS object at 0x7fd10c0f54a8>,)
F <function DUCKS.count at 0x7fd10c132f28> count
result: 32
dict: {'_cache': {('count', ()): 32}, 'ducks': 32}

---COUNT---
dict: {'_cache': {('count', ()): 32}, 'ducks': 32}
args (<__main__.DUCKS object at 0x7fd10c0f54a8>,)
F <function DUCKS.count at 0x7fd10c132f28> count
result: 32
dict: {'_cache': {('count', ()): 32}, 'ducks': 32}
